# Fresh take

## 1. Initalize sites

In [1]:
%load_ext autoreload
import os
import os
import shutil
import dbPipeline
from parseFiles.helperFunctions.loadDict import loadDict

In [2]:
data_dump = r'U:\data-dump'
data_dump = 'example_data'

In [5]:
%autoreload 2
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass
db = dbPipeline.database(projectPath=projectPath,verbose=False,enableParallel=False,template=False)
Sites = loadDict(r'example_data\projectInventory_Template.yml')
# import yaml
# print(yaml.safe_dump(Sites))

db.projectInventory(newSites=Sites)

template = loadDict(r'example_data\source_Template.yml')

# siteID,measurementID = 'SCL','Met'

# dpath = r'SCL\2024\20240912'
# template[siteID][measurementID]['rootPath'] = os.path.join(data_dump,dpath)
# print(template[siteID][measurementID]['rootPath'])
# db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])

dpath = r'SCL\2024\20240912'
siteID,measurementID = 'SCL','Flux'
for siteID in template.keys():
    for measurementID in template[siteID].keys():
        template[siteID][measurementID]['rootPath'] = os.path.join(data_dump,dpath)
        db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])

W135 15.1144
parseCoordinates(ID='SCL', latitude=69.2264167, longitude=-135.2519067)
W134 52 51.92
parseCoordinates(ID='FIL', latitude=69.3722778, longitude=-134.8810889)
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\POSIX_timestamp
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\Ux_mean
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\Pressure_mean
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\CH4_mole_fraction_mean
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\Uz_mean
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\Diagnostic_7700_mean
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\press_mean
Writing: c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux\2024\Diagnostic_CSAT_mean
Writing: c:\Users\j

In [41]:
print(template['SCL'].keys())

dict_keys(['Flux', 'Met', 'Metadata', 'SSM_Profile', 'WSM_Profile'])


In [ ]:
%autoreload 2

df = dbPipeline.databaseFolder(path=r'C:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Flux',Years=[2024])
df.dataOut.columns

In [ ]:
df.dataOut['CH4_density_mean'].dropna().plot()#.to_csv(r'C:\Users\jskeeter\OneDrive - NRCan RNCan\Documents\Presentations\CANCH4\Data\SCL_met.csv')

In [ ]:
from parseFiles.helperFunctions.loadDict import loadDict
# importlib.reload(dbPipeline)

# db = dbPipeline.database(projectPath=projectPath,verbose=False,enableParallel=False)
template = loadDict(r'example_data\source_Template.yml')
# # sourcePath = r'U:\EC_Backup\SCL_2024\20240724'
sourcePath = r'example_data'
print(template)
for siteID in template:
    for measurementID in template[siteID]:
        db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])